<a href="https://colab.research.google.com/github/katiezchang/UNEP/blob/main/ass8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import json
from typing import Optional, Dict, Any, List

import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF

import psycopg2
from psycopg2.extras import Json as PgJson
from dotenv import load_dotenv

load_dotenv()

# DB (Supabase Postgres – session pooler)
DB_USER = os.getenv("DB_USER", "postgres")
# SECURITY: Set DB_PASSWORD environment variable, do not hardcode passwords
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
DB_HOST = os.getenv("DB_HOST", "pooler.tulbxwdifnzquliytsog.supabase.co")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "postgres")
COUNTRIES_TABLE = "countries"

BUR_LISTING_URL = "https://unfccc.int/BURs"

BUR_PDF_DIR = "bur_pdfs"
os.makedirs(BUR_PDF_DIR, exist_ok=True)


In [26]:
def get_db_conn():
    return psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        sslmode="require",
    )


def get_country_row(country: str) -> Optional[Dict[str, Any]]:
    conn = get_db_conn()
    try:
        cur = conn.cursor()
        cur.execute(
            f"SELECT id, name, sections FROM {COUNTRIES_TABLE} WHERE name = %s",
            (country,),
        )
        row = cur.fetchone()
        if not row:
            return None
        row_id, name, sections = row
        if isinstance(sections, str):
            sections = json.loads(sections)
        elif sections is None:
            sections = {}
        return {"id": row_id, "name": name, "sections": sections}
    finally:
        conn.close()


def upsert_country_sections(country: str, new_sections: Dict[str, Dict[str, str]]):
    existing = get_country_row(country)
    conn = get_db_conn()
    try:
        cur = conn.cursor()
        if existing:
            merged = {**(existing["sections"] or {}), **new_sections}
            cur.execute(
                f"""
                UPDATE {COUNTRIES_TABLE}
                SET sections = %s
                WHERE id = %s
                """,
                (PgJson(merged), existing["id"]),
            )
            print(f"[DB] Updated sections for {country} (id={existing['id']}).")
        else:
            cur.execute(
                f"""
                INSERT INTO {COUNTRIES_TABLE} (name, sections)
                VALUES (%s, %s)
                RETURNING id
                """,
                (country, PgJson(new_sections)),
            )
            new_id = cur.fetchone()[0]
            print(f"[DB] Inserted new country row for {country} (id={new_id}).")

        conn.commit()
    finally:
        conn.close()


In [27]:
def fetch_bur_listing_page() -> BeautifulSoup:
    resp = requests.get(BUR_LISTING_URL)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")


def find_status_table(soup: BeautifulSoup):
    tables = soup.find_all("table")
    for t in tables:
        text = t.get_text(" ", strip=True)
        if (
            "Status of BUR submissions" in text
            or "Status of submission of biennial update reports" in text
        ):
            return t
    return tables[-1] if tables else None


def normalize_country_name_for_match(name: str) -> str:
    return re.sub(r"[\s\-]", "", name.lower())


def get_latest_bur_link_for_country(
    country: str, soup: Optional[BeautifulSoup] = None
) -> Optional[str]:
    if soup is None:
        soup = fetch_bur_listing_page()

    table = find_status_table(soup)
    if table is None:
        print("[SCRAPER] Could not find BUR status table.")
        return None

    target_norm = normalize_country_name_for_match(country)

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if not cols:
            continue

        party_text = cols[0].get_text(strip=True)
        party_norm = normalize_country_name_for_match(party_text)

        if party_norm == target_norm:
            latest_link = None
            latest_label = None

            for idx, col in enumerate(cols[1:], start=1):
                a = col.find("a", href=True)
                if a:
                    href = a["href"]
                    latest_link = href
                    latest_label = f"BUR{idx}"

            if latest_link:
                if latest_link.startswith("/"):
                    latest_link = "https://unfccc.int" + latest_link
                print(
                    f"[SCRAPER] Latest BUR for {country}: {latest_label} -> {latest_link}"
                )
                return latest_link

    print(f"[SCRAPER] No BUR link found for {country} on listing page.")
    return None


In [ ]:
#  PART 2!!!!!!

def normalize_country_for_filename(country: str) -> str:
    return re.sub(r"[\s\-]", "_", country.upper())


def download_bur_pdf(url: str, country: str) -> str:
    country_norm = normalize_country_for_filename(country)
    filename = f"{country_norm}_BUR_latest.pdf"
    path = os.path.join(BUR_PDF_DIR, filename)

    resp = requests.get(url)
    resp.raise_for_status()
    with open(path, "wb") as f:
        f.write(resp.content)

    print(f"[DOWNLOAD] Saved {country} BUR -> {path}")
    return path


def get_or_download_bur_pdf(country: str, bur_url: str) -> str:
    country_norm = normalize_country_for_filename(country)
    path = os.path.join(BUR_PDF_DIR, f"{country_norm}_BUR_latest.pdf")
    if os.path.exists(path):
        print(f"[DOWNLOAD] Using cached BUR for {country}: {path}")
        return path
    return download_bur_pdf(bur_url, country)


def load_pdf_text(path: str) -> str:
    if not os.path.exists(path):
        raise FileNotFoundError(f"PDF not found: {path}")
    doc = fitz.open(path)
    pages = [page.get_text("text") for page in doc]
    return "\n".join(pages)


In [30]:
def extract_between(
    text: str, start_patterns: List[str], end_patterns: Optional[List[str]] = None
) -> str:
    start_re = re.compile("|".join(start_patterns), re.IGNORECASE)
    m_start = start_re.search(text)
    if not m_start:
        return ""

    start_idx = m_start.end()

    if end_patterns:
        end_re = re.compile("|".join(end_patterns), re.IGNORECASE)
        m_end = end_re.search(text, start_idx)
        end_idx = m_end.start() if m_end else len(text)
    else:
        end_idx = len(text)

    snippet = text[start_idx:end_idx].strip()
    return snippet


def extract_climate_transparency(text: str, country: str) -> str:
    start_patterns = [
        rf"Climate transparency in {re.escape(country)}",
        r"Climate transparency in the country",
        r"Climate transparency",
        r"Progress in the four modules of the Enhanced Transparency Framework",
    ]
    end_patterns = [
        r"National transparency framework",
        r"Baseline",
        r"Official reports? to the UNFCCC",
        r"Official reporting to the UNFCCC",
        r"\n[A-Z][A-Za-z ]{6,}\n",
    ]
    return extract_between(text, start_patterns, end_patterns)


def extract_official_reporting(text: str) -> str:
    start_patterns = [
        r"Official reports? to the UNFCCC",
        r"Official reporting to the UNFCCC",
        r"Reports submitted to the UNFCCC",
        r"Table\s*\d+\.?\s*Official reports to the UNFCCC",
    ]
    end_patterns = [
        r"Progress in the four modules of the Enhanced Transparency Framework",
        r"Progress in the four modules",
        r"Greenhouse gas inventory module",
        r"GHG inventory module",
        r"\n[A-Z][A-Za-z ]{6,}\n",
    ]
    return extract_between(text, start_patterns, end_patterns)


def extract_key_barriers(text: str) -> str:
    start_patterns = [
        r"Key barriers",
        r"Main barriers",
        r"Constraints and gaps",
        r"Constraints, gaps and needs",
        r"Challenges and gaps",
        r"Barriers to enhanced transparency",
    ]
    end_patterns = [
        r"Progress in the four modules",
        r"Greenhouse gas inventory module",
        r"Adaptation and vulnerability module",
        r"\n[A-Z][A-Za-z ]{6,}\n",
    ]
    return extract_between(text, start_patterns, end_patterns)


def build_sections_payload(text: str, country: str) -> Dict[str, Dict[str, str]]:
    climate = extract_climate_transparency(text, country)
    official = extract_official_reporting(text)
    barriers = extract_key_barriers(text)

    sections: Dict[str, Dict[str, str]] = {}

    if climate:
        sections["ClimateTransparency"] = {
            "doc_type": "BUR",
            "text": climate,
        }
    if official:
        sections["OfficialReportingUNFCCC"] = {
            "doc_type": "BUR",
            "text": official,
        }
    if barriers:
        sections["KeyBarriers"] = {
            "doc_type": "BUR",
            "text": barriers,
        }

    return sections


In [31]:
def country_has_all_sections(country: str) -> bool:
    row = get_country_row(country)
    if not row:
        return False

    sections = row["sections"] or {}
    needed = {"ClimateTransparency", "OfficialReportingUNFCCC", "KeyBarriers"}
    have = {k for k, v in sections.items() if v}
    missing = needed - have
    if missing:
        print(f"[DB] {country} is missing sections: {missing}")
        return False
    print(f"[DB] {country} already has all three sections, skipping extraction.")
    return True


def process_country(country: str):
    print("\n==============================")
    print("Processing country:", country)
    print("==============================")

    # 1. Skip if DB already has everything
    if country_has_all_sections(country):
        return

    # 2. Find latest BUR link
    soup = fetch_bur_listing_page()
    bur_url = get_latest_bur_link_for_country(country, soup)
    if not bur_url:
        print(f"[ERROR] Could not find BUR URL for {country}")
        return

    # 3. Download or reuse cached PDF (Part 2)
    pdf_path = get_or_download_bur_pdf(country, bur_url)

    # 4. Extract sections from PDF text
    text = load_pdf_text(pdf_path)
    sections_payload = build_sections_payload(text, country)

    if not sections_payload:
        print(
            f"[WARN] No sections extracted for {country}. Refine regex patterns if needed."
        )
        return

    # 5. Upsert into Supabase `countries.sections`
    upsert_country_sections(country, sections_payload)
    print(f"[DONE] Processed {country}. Added sections: {list(sections_payload.keys())}")


In [32]:
if __name__ == "__main__":
    countries_to_process = ["Cuba", "Jordan", "Guinea-Bissau"]
    for c in countries_to_process:
        process_country(c)



Processing country: Cuba


OperationalError: could not translate host name "pooler.tulbxwdifnzquliytsog.supabase.co" to address: Name or service not known
